# chord2vecを入力にcapo予測

In [1]:
from pathlib import Path
import sys,os
sys.path.append(os.pardir)
from tools.preprocess.common import CommonPreprocessor
from tools.preprocess.interaction_matrix_generator import InteractionMatrixGenerator
from tools.preprocess.bow_vectorizer import BOWVectorizer
from scipy.sparse import csr_matrix

In [168]:
orignal_path=Path("../data/preprocessed_50k.txt")
bow_threshold=100

In [169]:
cp=CommonPreprocessor(rare_capo_list=['capo6', 'capo7', "whole_down"],test_rate=0.2,split_seed=0)
songs=cp.get_song_list(orignal_path,shuffle=True)
songs=cp.remove_rare_capo_song(songs)
songs_train, songs_test=cp.split_dataset(songs,shuffle=False)

In [170]:
chord_stat_train=cp.retrieve_chord_stat(songs_train)
capo_stat_train=cp.retrieve_capo_stat(songs_train)

In [171]:
capo_stat_train

Counter({'capo0': 15713,
         'capo1': 5204,
         'capo2': 6402,
         'capo3': 5271,
         'capo4': 3831,
         'capo5': 2294,
         'half_down': 1190})

In [172]:
from sklearn.preprocessing import LabelEncoder
from sklearn.base import TransformerMixin

class LabelGenarater(TransformerMixin):
    def __init__(self):
        self.encoder= LabelEncoder()
    
    def fit(self,songs):
        self.encoder.fit([song["rec_capo"] for song in songs])
        return self
    
    def transform(self,songs):
        return self.encoder.transform([song["rec_capo"] for song in songs])

In [173]:
def evaluate(model,X_train,y_train,X_test,y_test):
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    return accuracy_score(y_test,y_pred)

In [174]:
lg=LabelGenarater()
y_train=lg.fit_transform(songs_train)
y_test=lg.fit_transform(songs_test)

In [175]:
vectorizer=BOWVectorizer(chord_stat=chord_stat_train,threshold=bow_threshold)

In [176]:
X_train_bow=vectorizer.get_chord_features(songs_train)
X_test_bow=vectorizer.get_chord_features(songs_test)
X_train_bow.shape,X_test_bow.shape

((39905, 335), (9977, 335))

In [177]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import accuracy_score

lgbm_cls = lgb.LGBMClassifier(objective='multiclass',num_class= 7)
evaluate(lgbm_cls,X_train_bow,y_train,X_test_bow,y_test)

/Users/hikaru/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.720056129096923

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
evaluate(lr,X_train_bow,y_train,X_test_bow,y_test)

## Chord2Vecモデル

In [ ]:
from gensim.models import word2vec

class Chord2VecAvg:
    def __init__(self,model_name,vectorizer):
        self.c2v = word2vec.Word2Vec.load(model_name)
        self.bow_vectorizer=vectorizer 
        self.embeddings= self.create_embedding_mat()
    
    def transform(self,songs):
        bow=self.bow_vectorizer.get_chord_features(songs)
        return bow @ self.embeddings / np.expand_dims(np.sum(bow,axis=1),axis=1)
        
    def create_embedding_mat(self):
        embedding_dim= len(self.c2v.wv.vectors[0])
        embeddings=np.zeros((len(self.bow_vectorizer.chord_encoder),embedding_dim))
        for chord, index in self.bow_vectorizer.chord_encoder.items():
            embeddings[index] = self.c2v.wv[chord]
        return embeddings

In [ ]:
embedding_dim=100
min_count=5
window_size=5
iter_num=100
model_name=f"../result/w2v/{embedding_dim}_{min_count}_{window_size}_{iter_num}.model"

c2v_vectorizer=Chord2VecAvg(model_name,vectorizer)

In [ ]:
X_train_c2v=c2v_vectorizer.transform(songs_train)
X_test_c2v=c2v_vectorizer.transform(songs_test)
X_train_c2v.shape,X_test_c2v.shape

In [ ]:
lgbm_cls = lgb.LGBMClassifier(objective='multiclass',num_class= 7)
evaluate(lgbm_cls,X_train_c2v,y_train,X_test_c2v,y_test)

In [ ]:
lr = LogisticRegression()
evaluate(lr,X_train_c2v,y_train,X_test_c2v,y_test)